<a href="https://colab.research.google.com/github/yenikapatitatwik/Satellite_Imagery_Python/blob/master/crops_ndvi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentinelsat
!pip install geopandas
!pip install rasterio
!pip install matplotlib

     |████████████████████████████████| 972kB 5.5MB/s 
     |████████████████████████████████| 14.8MB 309kB/s 
     |████████████████████████████████| 6.5MB 48.7MB/s 
     |████████████████████████████████| 18.3MB 1.3MB/s 


In [14]:
from sentinelsat import SentinelAPI
from datetime import date
import geopandas as gpd
import folium 

user = 'tatwik' 
password = 'taTwik@2020' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

shape_file = gpd.read_file('/content/drive/MyDrive/crops/layers_2/POLYGON.shp')

map_4 = folium.Map([15.701783333333333,80.07289722222222], zoom_start=10)
folium.GeoJson(shape_file).add_to(map_4)
map_4.save("map_new.html")

footprint = None
for i in shape_file['geometry']:
  # print(i)
  footprint = i

products = api.query(footprint,
                     date = ('20201201', '20210120'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,5)
                    )
len(products)

products_gdf = api.to_geodataframe(products)

products_gdf_sorted = products_gdf.sort_values(['beginposition'], ascending=[True])
products_gdf_sorted

# api.download('cc8030d8-2338-49da-9343-fd050621b875')

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,orbitnumber,relativeorbitnumber,vegetationpercentage,notvegetatedpercentage,waterpercentage,unclassifiedpercentage,mediumprobacloudspercentage,highprobacloudspercentage,snowicepercentage,cloudcoverpercentage,level1cpdiidentifier,format,processingbaseline,platformname,filename,instrumentname,instrumentshortname,size,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,uuid,geometry
8725e99b-5d45-4dc7-b79b-a1a1b2b6a89c,S2A_MSIL2A_20201221T050221_N0214_R119_T44PMC_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-12-21T05:02:21.024Z, Instrument: MS...",false,2020-12-21 05:02:21.024,2020-12-21 05:02:21.024,2020-12-21 11:28:29.385,28716,119,21.828610,17.237633,46.782762,8.939327,0.171287,0.327096,0.000176,1.663923,S2A_OPER_MSI_L1C_TL_EPAE_20201221T061049_A0287...,SAFE,02.14,Sentinel-2,S2A_MSIL2A_20201221T050221_N0214_R119_T44PMC_2...,Multi-Spectral Instrument,MSI,976.61 MB,GS2A_20201221T050221_028716_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20201221T050221_N0214_R119_T44PMC_2...,8725e99b-5d45-4dc7-b79b-a1a1b2b6a89c,"MULTIPOLYGON (((80.06824 15.28624, 81.09091 15..."
cc8030d8-2338-49da-9343-fd050621b875,S2A_MSIL2A_20201221T050221_N0214_R119_T44PLC_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-12-21T05:02:21.024Z, Instrument: MS...",false,2020-12-21 05:02:21.024,2020-12-21 05:02:21.024,2020-12-21 11:27:16.859,28716,119,53.398216,16.354822,4.367801,21.671924,0.276970,0.122881,0.000000,0.403301,S2A_OPER_MSI_L1C_TL_EPAE_20201221T061049_A0287...,SAFE,02.14,Sentinel-2,S2A_MSIL2A_20201221T050221_N0214_R119_T44PLC_2...,Multi-Spectral Instrument,MSI,277.51 MB,GS2A_20201221T050221_028716_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20201221T050221_N0214_R119_T44PLC_2...,cc8030d8-2338-49da-9343-fd050621b875,"MULTIPOLYGON (((79.82302 15.28457, 80.15989 15..."
3e226115-bc19-47ec-99f5-15845d6348e0,S2A_MSIL2A_20201224T051221_N0214_R019_T44PMC_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-12-24T05:12:21.024Z, Instrument: MS...",false,2020-12-24 05:12:21.024,2020-12-24 05:12:21.024,2020-12-24 12:27:53.799,28759,19,43.502709,8.239539,1.270849,29.912573,1.985939,2.945631,0.000022,4.933197,S2A_OPER_MSI_L1C_TL_EPAE_20201224T062712_A0287...,SAFE,02.14,Sentinel-2,S2A_MSIL2A_20201224T051221_N0214_R019_T44PMC_2...,Multi-Spectral Instrument,MSI,206.66 MB,GS2A_20201224T051221_028759_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20201224T051221_N0214_R019_T44PMC_2...,3e226115-bc19-47ec-99f5-15845d6348e0,"MULTIPOLYGON (((80.06824 15.28624, 80.10444 15..."
2ef53ac1-5af8-4a65-bc6a-36bea5641ab6,S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-01-03T05:12:21.024Z, Instrument: MS...",false,2021-01-03 05:12:21.024,2021-01-03 05:12:21.024,2021-01-03 10:46:58.248,28902,19,22.589086,4.686286,1.232976,67.255855,2.715822,0.647701,0.001010,3.363523,S2A_OPER_MSI_L1C_TL_VGS1_20210103T070520_A0289...,SAFE,02.14,Sentinel-2,S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_2...,Multi-Spectral Instrument,MSI,95.11 MB,GS2A_20210103T051221_028902_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_2...,2ef53ac1-5af8-4a65-bc6a-36bea5641ab6,"MULTIPOLYGON (((80.06824 15.28624, 80.09925 15..."


In [15]:
from sentinelsat import SentinelAPI
from datetime import date
import geopandas as gpd
import folium 

user = 'tatwik' 
password = 'taTwik@2020' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

api.download('2ef53ac1-5af8-4a65-bc6a-36bea5641ab6')

Downloading: 100%|██████████| 99.8M/99.8M [00:06<00:00, 15.6MB/s]
MD5 checksumming: 100%|██████████| 99.8M/99.8M [00:00<00:00, 412MB/s]


{'Creation Date': datetime.datetime(2021, 1, 3, 10, 47, 6, 536000),
 'Ingestion Date': datetime.datetime(2021, 1, 3, 10, 46, 58, 248000),
 'Online': True,
 'date': datetime.datetime(2021, 1, 3, 5, 12, 21, 24000),
 'downloaded_bytes': 99788535,
 'footprint': 'POLYGON((80.06494624264963 16.006457070330764,80.25868008842201 15.953158204905446,80.22331100384912 15.805754312828398,80.18788996031155 15.657465042009898,80.15251819742653 15.50912800197378,80.11710434526788 15.360752387097213,80.09925080810693 15.286295734994463,80.06824451551739 15.286237505336244,80.06494624264963 16.006457070330764))',
 'id': '2ef53ac1-5af8-4a65-bc6a-36bea5641ab6',
 'md5': 'B158A64A3E9A2E82F03F219E2B2C4F20',
 'path': './S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_20210103T072845.zip',
 'size': 99788535,
 'title': 'S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_20210103T072845',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('2ef53ac1-5af8-4a65-bc6a-36bea5641ab6')/$value"}

In [16]:
import zipfile
with zipfile.ZipFile("/content/S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_20210103T072845.zip", 'r') as zip_ref:
    zip_ref.extractall("S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_20210103T072845")

In [13]:
from sentinelsat import SentinelAPI
from datetime import date
import geopandas.plotting as plot
import geopandas as gpd
import folium 
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
import rasterio.mask as rm
import rasterio.plot as rp
import matplotlib.pyplot as plt
import numpy as np

shape_file = gpd.read_file('/content/drive/MyDrive/crops/layers_2/POLYGON.shp') 
print(shape_file.crs)
R10 = '/content/drive/MyDrive/crops/S2A_MSIL2A_20201221T050221_N0214_R119_T44PLC_20201221T070610/S2A_MSIL2A_20201221T050221_N0214_R119_T44PLC_20201221T070610.SAFE/GRANULE/L2A_T44PLC_A028716_20201221T050221/IMG_DATA/R10m'
b4 = rio.open(R10+'/T44PLC_20201221T050221_B04_10m.jp2',driver='JP2OpenJPEG')
b3 = rio.open(R10+'/T44PLC_20201221T050221_B03_10m.jp2',driver='JP2OpenJPEG')
b2 = rio.open(R10+'/T44PLC_20201221T050221_B02_10m.jp2',driver='JP2OpenJPEG')
b8 = rio.open(R10+'/T44PLC_20201221T050221_B08_10m.jp2')
print(b4.crs)
### Create an RGB image ###
with rio.open('RGB_6.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b4.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b2.read(1),3) 
    rgb.close()

### Reproject Shape_file ###
shape_proj = shape_file.to_crs({'init': 'epsg:32644'})

### Clipping ###
with rio.open("RGB_6.tiff") as src:
    out_image, out_transform = rio.mask.mask(src, shape_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rio.open("RGB_6_masked.tiff", "w", **out_meta) as dest:
  dest.write(out_image)

red = b4.read()
nir = b8.read()

# # Calculate ndvi
ndvi = (nir-red)/(nir+red)

# Write the NDVI image
meta = b4.meta
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)

with rio.open('NDVI_6.tiff', 'w', **meta) as dst:
    dst.write(ndvi.astype(rio.float32))

with rio.open("NDVI_6.tiff") as src:
    out_image, out_transform = rio.mask.mask(src, shape_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rio.open("NDVI_6_masked.tiff", "w", **out_meta) as dest:
  dest.write(out_image)


epsg:4326
EPSG:32644


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide


In [17]:
from sentinelsat import SentinelAPI
from datetime import date
import geopandas.plotting as plot
import geopandas as gpd
import folium 
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
import rasterio.mask as rm
import rasterio.plot as rp
import matplotlib.pyplot as plt
import numpy as np

shape_file = gpd.read_file('/content/drive/MyDrive/crops/layers_2/POLYGON.shp') 
print(shape_file.crs)
R10 = '/content/S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_20210103T072845/S2A_MSIL2A_20210103T051221_N0214_R019_T44PMC_20210103T072845.SAFE/GRANULE/L2A_T44PMC_A028902_20210103T052419/IMG_DATA/R10m'
b4 = rio.open(R10+'/T44PMC_20210103T051221_B04_10m.jp2',driver='JP2OpenJPEG')
b3 = rio.open(R10+'/T44PMC_20210103T051221_B03_10m.jp2',driver='JP2OpenJPEG')
b2 = rio.open(R10+'/T44PMC_20210103T051221_B02_10m.jp2',driver='JP2OpenJPEG')
b8 = rio.open(R10+'/T44PMC_20210103T051221_B08_10m.jp2')
print(b4.crs)
### Create an RGB image ###
with rio.open('RGB_7.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b4.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b2.read(1),3) 
    rgb.close()

### Reproject Shape_file ###
shape_proj = shape_file.to_crs({'init': 'epsg:32644'})

### Clipping ###
with rio.open("RGB_7.tiff") as src:
    out_image, out_transform = rio.mask.mask(src, shape_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rio.open("RGB_7_masked.tiff", "w", **out_meta) as dest:
  dest.write(out_image)

red = b4.read()
nir = b8.read()

# # Calculate ndvi
ndvi = (nir-red)/(nir+red)

# Write the NDVI image
meta = b4.meta
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)

with rio.open('NDVI_7.tiff', 'w', **meta) as dst:
    dst.write(ndvi.astype(rio.float32))

with rio.open("NDVI_7.tiff") as src:
    out_image, out_transform = rio.mask.mask(src, shape_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rio.open("NDVI_7_masked.tiff", "w", **out_meta) as dest:
  dest.write(out_image)


epsg:4326
EPSG:32644


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide
